In [1]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO

In [2]:
test_dir = "./using_phone_frames"

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

In [3]:
def extract_images_from_video(
    video_path: str,
    output_folder: str,
    time_step: float = 0.25,
    is_rotate: bool = False,
):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Lấy tên file gốc không bao gồm phần mở rộng
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"Frames per second using video.get(cv2.CAP_PROP_FPS): {fps}")

    frame_interval = int(fps * time_step)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if is_rotate:
            frame = cv2.rotate(
                frame, cv2.ROTATE_90_COUNTERCLOCKWISE
            )  # Xoay 90 độ ngược chiều kim đồng hồ

        if frame_count % frame_interval == 0:
            saved_count += 1
            # Định dạng tên file: <tên file gốc>_<số thứ tự dạng 3 chữ số>.jpg

            output_file = os.path.join(
                output_folder, f"{base_name}_{saved_count:03d}.jpg"
            )
            cv2.imwrite(output_file, frame)
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

In [4]:
extract_images_from_video("./test_data.mp4", test_dir, time_step=0.25, is_rotate=False)

Frames per second using video.get(cv2.CAP_PROP_FPS): 0.0


### Compare accuracy of Yolo8n and Yolo11n, Yolo11s

In [5]:
images = []
model_accuracies = []

for filename in os.listdir(test_dir):
    if filename.endswith(".jpg"):
        images.append(os.path.join(test_dir, filename))

In [6]:
def analyze_phone_detection(model_name):
    model = YOLO(model_name)
    undetected_phone_images = []
    image_count = len(images)

    for image in images:
        results = model(image, classes=[67], imgsz=640,verbose=False)

        if len(results[0].boxes) == 0:
            undetected_phone_images.append(image)

    print(f"Total images: {image_count}")
    print(f"Images without phone detected: {len(undetected_phone_images)}")
    print(f"Images with phone detected: {image_count - len(undetected_phone_images)}")

    accuracy = (image_count - len(undetected_phone_images)) / image_count * 100
    model_accuracies.append({
        "model_name": model_name.split(".")[0],
        "accuracy": accuracy,
    })
    print(f"Accuracy: {accuracy:.2f}%")

In [7]:
analyze_phone_detection ("yolov5nu.pt")

Total images: 166
Images without phone detected: 94
Images with phone detected: 72
Accuracy: 43.37%


In [8]:
analyze_phone_detection ("yolov8n.pt")

Total images: 166
Images without phone detected: 48
Images with phone detected: 118
Accuracy: 71.08%


In [9]:
analyze_phone_detection ("yolov8s.pt")

Total images: 166
Images without phone detected: 86
Images with phone detected: 80
Accuracy: 48.19%


In [10]:
analyze_phone_detection("yolov9t.pt")

Total images: 166
Images without phone detected: 108
Images with phone detected: 58
Accuracy: 34.94%


In [11]:
analyze_phone_detection ("yolo11n.pt")

Total images: 166
Images without phone detected: 67
Images with phone detected: 99
Accuracy: 59.64%


In [12]:
analyze_phone_detection ("yolo11s.pt")

Total images: 166
Images without phone detected: 25
Images with phone detected: 141
Accuracy: 84.94%


In [13]:
yolo_evaluation = pd.read_csv("../../tool/detect_cell_phone/detection_speed2.csv")

In [14]:
yolo_evaluation

,Model,Average Inference Time (ms)
0,yolov5nu,53.688467
1,yolov8n,47.938562
2,yolov8s,113.835681
3,yolov9t,82.349341
4,yolo11n,58.184477
5,yolo11s,114.807193


In [15]:
# Thêm cột Accuracy vào DataFrame
yolo_evaluation["Accuracy"] = None

# Cập nhật giá trị Accuracy cho từng model
for i, row in yolo_evaluation.iterrows():
    for accuracy in model_accuracies:
        if row["Model"] == accuracy["model_name"]:
            yolo_evaluation.at[i, "Accuracy"] = accuracy["accuracy"]

In [16]:
yolo_evaluation

,Model,Average Inference Time (ms),Accuracy
0,yolov5nu,53.688467,43.373494
1,yolov8n,47.938562,71.084337
2,yolov8s,113.835681,48.192771
3,yolov9t,82.349341,34.939759
4,yolo11n,58.184477,59.638554
5,yolo11s,114.807193,84.939759


In [17]:
yolo_evaluation.to_csv("../../tool/detect_cell_phone/detection_speed2.csv", index=False)